In [1]:
from test import EigenData
import numpy as np
import scipy.io
import h5py
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
%matplotlib inline

In [2]:
data = EigenData()
eigenworms = data.get_eigenworms('EigenWorms.mat')

footage = data.get_footage('20150814-All-PNAS2011-DataStitched .mat')
newdata = torch.rand(33600,5)
for k in footage.keys():
    print(k, footage[k].shape)
    arr1=footage[k]

    arr2=arr1[:5,]

    r = data.reconstruct(footage[k])
    print('-->', r.shape)
    dataset1 = arr2.transpose()

    newdata = np.concatenate((newdata,dataset1),axis=0)
newdata = newdata[33600:,]
print(newdata.shape)

b (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
c (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
d (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
e (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
f (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
g (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
h (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
i (6, 33600)
(100, 6) (6, 33600)
--> (100, 33600)
j (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
k (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
l (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
m (5, 33600)
(100, 5) (5, 33600)
--> (100, 33600)
(403200, 5)


In [3]:
data1=newdata[~np.isnan(newdata).any(axis=1)]

data1 = data1.astype('float32')
print(data1.shape)
print(len(data1))

(386422, 5)
386422


In [4]:
def create_dataset(data,n_predictions,n_next):
    
   
    dim = data.shape[1]
    train_X, train_Y = [], []
    for i in range(data.shape[0]-n_predictions-n_next-1):
        a = data[i:(i+n_predictions),:]
        train_X.append(a)
        tempb = data[(i+n_predictions):(i+n_predictions+n_next),:]
        b = []
        for j in range(len(tempb)):
            for k in range(dim):
                b.append(tempb[j,k])
        train_Y.append(b)
    train_X = np.array(train_X,dtype='float64')
    train_Y = np.array(train_Y,dtype='float64')

    return train_X, train_Y

In [5]:
X, Y = create_dataset(data1,5,5)

print(X.shape)
print(Y.shape)

(386411, 5, 5)
(386411, 25)


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
data1 = scaler.fit_transform(data1)
Y = scaler.fit_transform(Y)

In [7]:
data_X, data_Y = create_dataset(data1,5,5)

print(data_X.shape)
print(data_Y.shape)

(386411, 5, 5)
(386411, 25)


In [8]:
train_size = int(len(data_X) * 0.8)
test_size = len(data_X) - train_size
train_X = data_X[:train_size]
train_Y = Y[:train_size]
test_X = data_X[train_size:]
test_Y = Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(309128, 5, 5)
(309128, 25)
(77283, 5, 5)
(77283, 25)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input,LSTM,Dropout,Dense


timesteps = 5
# create and fit the LSTM network

model = keras.Sequential()
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100,input_shape=(timesteps,5),return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25,activation='linear'))
model.compile(optimizer='rmsprop',loss='mean_squared_error',metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(train_X, train_Y, epochs=100, batch_size=512, verbose=2, validation_data=(test_X, test_Y))

Epoch 1/100
604/604 - 32s - loss: 0.0131 - root_mean_squared_error: 0.1144 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0684
Epoch 2/100
604/604 - 29s - loss: 0.0050 - root_mean_squared_error: 0.0705 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0434
Epoch 3/100
604/604 - 27s - loss: 0.0030 - root_mean_squared_error: 0.0545 - val_loss: 0.0013 - val_root_mean_squared_error: 0.0357
Epoch 4/100
604/604 - 27s - loss: 0.0022 - root_mean_squared_error: 0.0466 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0330
Epoch 5/100
604/604 - 28s - loss: 0.0018 - root_mean_squared_error: 0.0420 - val_loss: 8.2619e-04 - val_root_mean_squared_error: 0.0287
Epoch 6/100
604/604 - 27s - loss: 0.0015 - root_mean_squared_error: 0.0390 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0316
Epoch 7/100
604/604 - 28s - loss: 0.0013 - root_mean_squared_error: 0.0367 - val_loss: 6.7092e-04 - val_root_mean_squared_error: 0.0259
Epoch 8/100
604/604 - 30s - loss: 0.0012 - root_mean_squared_error: 

Epoch 60/100
604/604 - 26s - loss: 6.6039e-04 - root_mean_squared_error: 0.0257 - val_loss: 3.5254e-04 - val_root_mean_squared_error: 0.0188
Epoch 61/100
604/604 - 27s - loss: 6.5933e-04 - root_mean_squared_error: 0.0257 - val_loss: 3.2287e-04 - val_root_mean_squared_error: 0.0180
Epoch 62/100
604/604 - 28s - loss: 6.5663e-04 - root_mean_squared_error: 0.0256 - val_loss: 3.3721e-04 - val_root_mean_squared_error: 0.0184
Epoch 63/100
604/604 - 26s - loss: 6.5492e-04 - root_mean_squared_error: 0.0256 - val_loss: 3.0230e-04 - val_root_mean_squared_error: 0.0174
Epoch 64/100
604/604 - 27s - loss: 6.5447e-04 - root_mean_squared_error: 0.0256 - val_loss: 3.0453e-04 - val_root_mean_squared_error: 0.0175
Epoch 65/100
604/604 - 27s - loss: 6.5278e-04 - root_mean_squared_error: 0.0255 - val_loss: 3.2400e-04 - val_root_mean_squared_error: 0.0180
Epoch 66/100
604/604 - 27s - loss: 6.4938e-04 - root_mean_squared_error: 0.0255 - val_loss: 3.0848e-04 - val_root_mean_squared_error: 0.0176
Epoch 67/100


In [10]:
testPredict = model.predict(test_X)

In [11]:
newtestPredict = scaler.inverse_transform(testPredict)
newtest_Y = scaler.inverse_transform(test_Y)

In [12]:
testScore = sqrt(mean_squared_error(newtest_Y, newtestPredict))
print('Test Score: %.5f RMSE' % (testScore))

Test Score: 0.45812 RMSE
